# SVM

### When to use SVM?
* Classification, Regression
* Use when number of features are more
* SVM uses best separating line possible, while other algorihtm just separate the points. When test data is used, the best separating line will give good result and not just any separating line.
* SVM kernel can be used for non linear separations

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline

In [2]:
df=pd.read_csv('F:/GitHub/Machine-Learning/SVM/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df.shape

(284807, 31)

In [4]:
#It is highly imbalanced dataset.
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

### Dealing with imbalanced dataset

### Down sampling
Reduce occurance of class having more occurance by choosing at random. Can be used when samples >10,000.


### Over sampling
Increase the count of less occuranced class by duplicating.


#### Note:
Code for both upsampling and downsampling is written. But here we are using down sampling because SVM works when number of features are more compared to data points.



In [5]:
df_majority = df[df['Class']==0]
df_minority = df[df['Class']==1]
print("df_majority=",df_majority.shape)
print("df_minority=",df_minority.shape)

df_majority= (284315, 31)
df_minority= (492, 31)


In [6]:
from sklearn.utils import resample
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),  # to match minority class
                                 random_state=1234) # reproducible results

print ("df_majority=",df_majority_downsampled.shape)
print("df_minority=",df_minority.shape)

df_majority= (492, 31)
df_minority= (492, 31)


In [7]:
##Uncomment this section for upsampling
#from sklearn.utils import resample
##Upsample minority class
#df_minority_upsampled = resample(df_minority, 
#                                 replace=True,    # sample without replacement
#                                 n_samples=len(df_majority),  # to match minority class
#                                 random_state=1234) # reproducible results
#print ("df_minority=",df_minority_upsampled.shape)
#print("df_majority=",df_majority.shape)

In [8]:
#combining equal number of data points
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
9279,13377.0,-1.887363,1.292940,-1.029589,-0.054906,-3.459512,1.249290,2.416203,0.752276,0.441309,...,0.001851,0.317728,0.261672,0.211446,-0.540258,0.243535,-0.154486,-0.258593,641.27,0
188496,127999.0,2.115468,-0.758276,-1.686924,-1.938542,-0.295361,-1.048351,-0.129282,-0.210317,2.589292,...,0.099975,0.650791,-0.008366,0.634501,0.432511,-0.663581,0.033601,-0.053899,7.99,0
267259,162694.0,2.026814,-0.417457,-1.414403,0.131006,0.207052,0.044228,-0.207533,0.029496,0.772332,...,0.067981,0.361787,0.022414,0.258958,0.097178,0.732207,-0.086327,-0.076725,18.95,0
97925,66442.0,-1.352638,0.398110,-0.077402,0.497205,0.808162,-1.665914,0.428428,0.160525,-0.508415,...,-0.107546,0.046688,0.797856,0.371495,-0.045342,0.249126,-0.035772,-0.272748,0.76,0
169899,119920.0,0.567238,0.902492,0.893143,4.960773,-0.842461,1.519354,-1.342256,-2.484131,-1.403265,...,-1.065843,0.960790,0.190154,-0.111327,0.747631,0.624810,-0.024930,0.067734,98.61,0


In [9]:
#Dropping Time
df_downsampled=df_downsampled.drop(['Time'],axis=1)

In [10]:
#separate features and target
X=df_downsampled.drop(['Class'],axis=1)
y=df_downsampled['Class']

In [11]:
#Feature Scaling (Optional in case of Naive Bayes)
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaler.fit(X)
X=scaler.fit_transform(X)
# As .fit changes to numpy. Therefore changing back to dataframe
#column_values=['age','workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
X=pd.DataFrame(X)

In [12]:
#Separate test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)
X_train.shape, X_test.shape

((738, 29), (246, 29))

In [13]:
from sklearn import svm
classifier = svm.SVC(kernel='linear')
classifier.fit(X_train, y_train)

#Predicting
y_pred = classifier.predict(X_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[126   2]
 [  9 109]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       128
           1       0.98      0.92      0.95       118

    accuracy                           0.96       246
   macro avg       0.96      0.95      0.96       246
weighted avg       0.96      0.96      0.96       246

